In [104]:
class MempoolTransaction():
  def __init__(self, txid, fee, weight, parents):
    self.txid = txid
    self.fee = int(fee)
    self.weight = int(weight)
    self.parents = parents 

In [105]:
def parse_mempool_csv():
  """Parse the CSV file and return a list of MempoolTransactions."""
  with open('mempool.csv') as f:
    return([MempoolTransaction(*line.strip().split(',')) for line in f.readlines()[1:]])

In [106]:
global MAXIMUM_BLOCK_WEIGHT 
MAXIMUM_BLOCK_WEIGHT = 4000000

In [107]:
def compute_helper_mempool_dictionaries():
  txn_density_dict = {}
  txn_parents_dict = {}
  txn_size_dict = {}
  mempool_data = parse_mempool_csv()
  for elem in mempool_data:
    size = elem.weight/MAXIMUM_BLOCK_WEIGHT # weight mapped to (0,1)
    txn_size_dict[elem.txid] = size 
    txn_density_dict[elem.txid] = elem.fee/size
    if elem.parents != '':
      txn_parents_dict[elem.txid] = elem.parents.strip().split(';')
  return txn_density_dict,txn_parents_dict,txn_size_dict

In [108]:
def order_TXN_density(txn_density_dict):
  sorted_txn_density = sorted(txn_density_dict.items(), key=lambda x: x[1], reverse=True)
  return sorted_txn_density

In [113]:
def create_valid_block(sorted_txn_density,txn_parents_dict,txn_size_dict):
  currentBlockSize = 0
  blockTxns = []
  while(len(sorted_txn_density) != 0):
    currentTxn = sorted_txn_density[0]
    if currentTxn[0] in txn_parents_dict: #has parents
      parents = txn_parents_dict[currentTxn[0]]
      totalParentsTxnSize = 0
      for parent in parents:
        totalParentsTxnSize = totalParentsTxnSize + txn_size_dict[parent]
      if currentBlockSize + totalParentsTxnSize > 1:
        sorted_txn_density.pop(0)
        continue

    #no parents
    if currentBlockSize + txn_size_dict[currentTxn[0]] >= 1:
      break
    else:
      currentBlockSize = currentBlockSize + txn_size_dict[currentTxn[0]]
    blockTxns.append(currentTxn[0])
    sorted_txn_density.pop(0)
  return currentBlockSize,blockTxns

In [ ]:
if __name__ == "__main__":
  txn_density_dict,txn_parents_dict,txn_size_dict = compute_helper_mempool_dictionaries()
  sorted_txn_density = order_TXN_density(txn_density_dict)
  print(len(sorted_txn_density))
  currentBlockSize,blockTxns = create_valid_block(sorted_txn_density,txn_parents_dict,txn_size_dict)